# Advanced Coupled Mode Equations models

This tutorial demonstrates how to use the more advanced CMEs models to simulate the nonlinear response of a TWPA.

## Initialize TWPAnalysis

First, let's load a KITWPA model and plot its reponse. Since we are going to consider higher order processes, we will initialize TWPAnalysis with a larger frequency span

In [ ]:
import logging

import matplotlib.pyplot as plt
import numpy as np

from twpasolver import TWPAnalysis
from twpasolver.logger import log
from twpasolver.mathutils import dBm_to_I

log.setLevel(logging.WARNING)

plt.rcParams["font.size"] = 13.5
plt.rcParams["axes.axisbelow"] = True

twpa_file = "model_cpw_dartwars_13nm_Lk8_5.json"
a = TWPAnalysis(twpa=twpa_file, f_arange=(0.2, 30, 0.5e-3))
for c in a.twpa.cells:
    c.delta = 0e-4
a.twpa.Z0_ref = 50
a.update_base_data()
optimal_pump = 7.4735
a.data["optimal_pump_freq"] = optimal_pump
ax = a.plot_response(pump_freq=optimal_pump)

# plot first and second pump harmonic
ax[0].axvline(2 * optimal_pump, ls="--", c="r")
ax[1].axvline(2 * optimal_pump, ls="--", c="r")
ax[0].axvline(3 * optimal_pump, ls="--", c="r")
ax[1].axvline(3 * optimal_pump, ls="--", c="r")
a.twpa.model_dump()
a.data["optimal_pump_freq"]

## ModeArrays and ModeArrayFactory
Properly describing the nonlinear response of TWPAs requires going beyond the basic CMEs system that considers only pump, signal and idler. ```twpasolver``` is able to automatically build and solve CMEs systems including an arbitrary number of physical modes connected by relations between their frequencies. A collection of related modes is represented in a ```ModeArray``` object, which has the following notable features:
- Organize the mode relations in a dependency graph, identifying the independent modes
- Automatically extrapolate and propagate the physical propertie of the modes (frequency, wavenumber...) according to the relations
- Automatically select the 3WM and 4WM terms that satisfy the Rotating Wave Approximation (RWA) when solving the CMEs

The ```ModeArray``` factory helps to set up custom mode arrays or create standard extended mode collections useful to improve the accuracy of your simulations.

Here is to setup a ModeArray with custom defined modes and relations

In [ ]:
from twpasolver.modes_rwa import ModeArrayFactory

mode_labels = ["p", "s", "i", "p2"]
mode_relations = [["i", "p-s"], ["p2", "p+p"]]
mode_array_p2 = ModeArrayFactory.create_custom(
    a.data,
    mode_labels=mode_labels,
    mode_directions=[1, 1, 1, 1],
    relations=mode_relations,
)
mode_array_p2.plot_mode_relations(layout="circular")

For obtaining more realistic simulations, you should consider pump harmonics, frequency conversion processes and signal/idler harmonics. The ```create_extended_3wm``` method helps you to do so. Let's also add this more complete set of mode to the analysis class

In [ ]:
mode_array = ModeArrayFactory.create_extended_3wm(
    a.data, n_pump_harmonics=2, n_frequency_conversion=2, n_signal_harmonics=1
)
mode_array.plot_mode_relations(show_frequencies=False)
a.add_mode_array("gain_extended", mode_array)

## Gain comparison with different models
You can use the attributes of the ```gain``` analysis function to specify which CME model and array of modes you want to include in the simulation. Once compied, even simulations with quite a large number of modes compared to the basic CMEs system should still be quite fast

In [ ]:
s_arange = np.arange(1, 6.5, 0.05)

res_gain_base = a.gain(signal_freqs=s_arange, Is0=1e-6, pump=optimal_pump)
ax = a.plot_gain()
res_gain_extended = a.gain(
    signal_freqs=s_arange,
    Is0=1e-6,
    pump=optimal_pump,
    model="general_ideal",
    mode_array_config="gain_extended",
)
res_gain_with_loss = a.gain(
    signal_freqs=s_arange,
    Is0=1e-6,
    pump=optimal_pump,
    model="general_loss_only",
    mode_array_config="gain_extended",
)

ax.plot(s_arange, res_gain_extended["gain_db"])
ax.plot(s_arange, res_gain_with_loss["gain_db"])
ax.legend(
    ["basic", "extended, ideal", "extended, with loss"], bbox_to_anchor=(1.1, 0.6)
)

The most realistic model takes into account both losses and reflections in the device, but it is more difficult to integrate for the ODE solver and usually takes around 10X more time to run than the other three models.

In [ ]:
res_gain_with_reflections_wide = a.gain(
    signal_freqs=s_arange,
    Is0=1e-6,
    pump=optimal_pump,
    model="general",
    mode_array_config="gain_extended",
)
ax = a.plot_gain()

In [ ]:
s_arange = np.arange(3, 3.5, 0.005)
res_gain_with_reflections_narrow = a.gain(
    signal_freqs=s_arange,
    Is0=1e-6,
    pump=optimal_pump,
    model="general",
    mode_array_config="gain_extended",
)
ax = a.plot_gain()

Another useful feature of ```TWPAnalysis``` is the ability to plot the average currents for each mode as a function of the position along the line. This is helpful to identify which processes are most relevant to the simulated response.

In [ ]:
a.plot_mode_currents()